# GPU (remote)

Let's start a worker on a gpu node of a supercomputer cluster.

In [50]:
using Distributed
# addprocs([("cbauer17@gpu2", 1)]; exename=`/projects/ag-trebst/bauer/bin/julia-1.3.1/bin/julia`, exeflags=`--project=/projects/ag-trebst/bauer/JuliaOulu20`, dir="/projects/ag-trebst/bauer/JuliaOulu20", tunnel=true)
addprocs([("cbauer17@gpu2", 1)]; exename=`/projects/ag-trebst/bauer/bin/julia-1.3.1/bin/julia`, exeflags=`--project=/projects/ag-trebst/bauer/JuliaOulu20/backup/gpu_remote`, dir="/projects/ag-trebst/bauer/JuliaOulu20/backup/gpu_remote", tunnel=true)
@fetch gethostname()

"cheops51801"

Extract the GPU name:

In [59]:
@fetch @eval using CUDAdrv
@fetch CUDAdrv.name(CuDevice(0))

"Tesla V100-SXM2-16GB"

## Benchmark matmul

In [77]:
@everywhere using CuArrays
@everywhere using BenchmarkTools

In [80]:
@fetch begin
    A, B = rand(1000,1000), rand(1000,1000);
    Agpu, Bgpu = CuArray(A), CuArray(B);
    
    println("Move array CPU -> GPU")
    @btime CuArray($A);
    
    println("A*B (cpu)")
    @btime $A * $B;

    println("A*B (gpu)")
    @btime $Agpu * $Bgpu;
    
    println("Move array GPU -> CPU")
    Cgpu = Agpu * Bgpu
    @btime Array($Cgpu);
    
    nothing
end

      From worker 7:	Move array CPU -> GPU
      From worker 7:	  1.812 ms (10 allocations: 7.63 MiB)
      From worker 7:	A*B (cpu)
      From worker 7:	  67.885 ms (2 allocations: 7.63 MiB)
      From worker 7:	A*B (gpu)
      From worker 7:	  12.952 μs (10 allocations: 416 bytes)
      From worker 7:	Move array GPU -> CPU
      From worker 7:	  1.094 ms (2 allocations: 7.63 MiB)


## Machine learning on the GPU

In [ ]:
@fetch Pkg.instantiate()

      From worker 7:	  Updating registry at `/projects/ag-trebst/bauer/.julia/registries/General`
      From worker 7:	  Updating git-repo `git://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /projects/ag-trebst/bauer/.julia/registries/General — failed to fetch from repo
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Pkg/src/Types.jl:1199


In [81]:
@everywhere using Flux

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1273


      From worker 7:	ERROR: LoadError: ArgumentError: Package Zygote [e88e6eb3-aa80-5325-afca-941959d7151f] is required but does not seem to be installed:
      From worker 7:	 - Run `Pkg.instantiate()` to install all recorded dependencies.
      From worker 7:	
      From worker 7:	Stacktrace:
      From worker 7:	 [1] _require(::Base.PkgId) at ./loading.jl:993
      From worker 7:	 [2] require(::Base.PkgId) at ./loading.jl:922
      From worker 7:	 [3] require(::Module, ::Symbol) at ./loading.jl:917
      From worker 7:	 [4] include at ./boot.jl:328 [inlined]
      From worker 7:	 [5] include_relative(::Module, ::String) at ./loading.jl:1105
      From worker 7:	 [6] include(::Module, ::String) at ./Base.jl:31
      From worker 7:	 [7] top-level scope at none:2
      From worker 7:	 [8] eval at ./boot.jl:330 [inlined]
      From worker 7:	 [9] eval(::Expr) at ./client.jl:425
      From worker 7:	 [10] top-level scope at ./none:3
      From worker 7:	in expression starting at /project

CompositeException: On worker 7:
Failed to precompile Flux [587475ba-b771-5e3f-ad9e-33799f191a9c] to /projects/ag-trebst/bauer/.julia/compiled/v1.3/Flux/QdkVy_9szkr.ji.
error at ./error.jl:33
compilecache at ./loading.jl:1283
_require at ./loading.jl:1024
require at ./loading.jl:922
#1 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Distributed/src/Distributed.jl:78
#105 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Distributed/src/process_messages.jl:290
run_work_thunk at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Distributed/src/process_messages.jl:79
run_work_thunk at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Distributed/src/process_messages.jl:88
#98 at ./task.jl:333

In [33]:
@fetch begin
    W = cu(rand(2, 5)) # a 2×5 CuArray
    
    b = cu(rand(2))

    predict(x) = W*x .+ b
    loss(x, y) = sum((predict(x) .- y).^2)

    x, y = cu(rand(5)), cu(rand(2)) # Dummy data
    loss(x, y) # ~ 3
end





10.323411f0

In [ ]:
# Flux.train!(loss, params(m), [(data,labels)], opt)

In [31]:
@fetch begin
#     m = Chain(
#     Dense(10, 5),
#     Dense(5, 2),
#     softmax # normalize output neurons
#     ) |> gpu
    
    
    data = rand(10, 100) |> gpu # fake data
    labels = fill(0.5, 2, 100) |> gpu # fake data
    
#     loss(x, y) = sum(Flux.mse(m(x), y)) # mean squared error
#     opt = Descent(0.01) # or ADAM
end

RemoteException: KeyError: key CuArrays [3a865a2d-5b23-5a0f-bc46-62713ec82fae] not found
getindex at .\dict.jl:477 [inlined]
root_module at .\loading.jl:962 [inlined]
deserialize_module(::Distributed.ClusterSerializer{Sockets.TCPSocket}) at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Serialization\src\Serialization.jl:894
handle_deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Int32) at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Serialization\src\Serialization.jl:799
deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}) at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Serialization\src\Serialization.jl:722
deserialize_datatype(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Bool) at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\Serialization\src\Serialization.jl:1192
handle_deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Int32) at D:\buildbot\worke